In [7]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 📊 Growth Accounting: Decomposing Economic Growth

Where does economic growth come from? Growth accounting is a technique used to break down the growth rate of total output ($Y$) into contributions from the growth rates of measurable inputs – typically capital ($K$) and labor ($L$) – and a residual component attributed to technological progress or efficiency gains, known as Total Factor Productivity (TFP) growth ($A$).

Assuming a Cobb-Douglas production function with constant returns to scale:
$$Y = A K^{\alpha} L^{1 - \alpha}$$

We can take logarithms and differentiate with respect to time (or approximate using discrete changes) to arrive at the growth accounting equation:

$$g_Y = \alpha g_K + (1 - \alpha) g_L + g_A$$

Where:
- $g_Y = \frac{\dot{Y}}{Y}$ is the growth rate of output.
- $g_K = \frac{\dot{K}}{K}$ is the growth rate of capital.
- $g_L = \frac{\dot{L}}{L}$ is the growth rate of labor.
- $g_A = \frac{\dot{A}}{A}$ is the growth rate of TFP (often called the Solow Residual).
- $\alpha$ is capital's share of income.

This equation tells us that output growth ($g_Y$) is a weighted average of the growth rates of capital and labor (weighted by their respective income shares), plus the growth rate of TFP.

# ⚙️ Simulation Setup: Growth by Decade

This simulation models an economy over several decades (e.g., 1980s, 1990s, 2000s), allowing the growth rates of capital ($g_K$), labor ($g_L$), and TFP ($g_A$) to differ across these periods.

1.  **Initialization:** We start with initial levels of $K$, $L$, and $A$ at the beginning of the first decade (e.g., 1980).
2.  **Decade-Specific Growth:** For each year within a decade, we apply the constant annual growth rates specified for that decade to calculate the levels of $K$, $L$, and $A$ for the following year.
    - $K_{t+1} = K_t (1 + g_{K, \text{decade}})$
    - $L_{t+1} = L_t (1 + g_{L, \text{decade}})$
    - $A_{t+1} = A_t (1 + g_{A, \text{decade}})$
3.  **Output Calculation:** Output ($Y_t$) in each year is calculated using the Cobb-Douglas function with the corresponding levels of $K_t$, $L_t$, and $A_t$.
4.  **Growth Accounting Decomposition:** After simulating the levels, we calculate the average annual growth rate of $Y$ within each decade and decompose it into the average contributions from $g_K$, $g_L$, and $g_A$ using the growth accounting equation.

The interactive sliders allow you to specify the initial levels and the decade-specific growth rates for $K$, $L$, and $A$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def dynamic_growth_accounting(alpha=0.33,
                               K80=1.0, L80=1.0, A80=1.0,
                               Kg1=0.03, Lg1=0.02, Ag1=0.01,
                               Kg2=0.05, Lg2=0.01, Ag2=0.015,
                               Kg3=0.02, Lg3=0.01, Ag3=0.007):

    decades = ['1980s', '1990s', '2000s']
    T = 10  # years per decade
    K = [K80]
    L = [L80]
    A = [A80]
    Y = []

    growth_data = [(Kg1, Lg1, Ag1), (Kg2, Lg2, Ag2), (Kg3, Lg3, Ag3)]

    for Kg, Lg, Ag in growth_data:
        for _ in range(T):
            K.append(K[-1] * (1 + Kg))
            L.append(L[-1] * (1 + Lg))
            A.append(A[-1] * (1 + Ag))
            Y.append(K[-2]**alpha * (A[-2] * L[-2])**(1 - alpha))

    K, L, A = np.array(K[:-1]), np.array(L[:-1]), np.array(A[:-1])
    Y = np.array(Y)

    # Compute log-level contributions
    log_Y = np.log(Y)
    contrib_K = alpha * np.log(K)
    contrib_L = (1 - alpha) * np.log(L)
    contrib_A = log_Y - contrib_K - contrib_L

    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    axs[0].plot(Y, label='Output Y(t)', color='darkgreen')
    axs[0].set_title('Total Output (Level)')
    axs[0].set_xlabel('Years Since 1980')
    axs[0].grid(True)
    axs[0].legend()

    axs[1].plot(log_Y, label='log Y', color='black', lw=2)
    axs[1].plot(contrib_K, label='Capital Contribution', color='dodgerblue')
    axs[1].plot(contrib_L, label='Labor Contribution', color='seagreen')
    axs[1].plot(contrib_A, label='TFP Contribution', color='orange')
    axs[1].set_title('Log Output and Component Contributions')
    axs[1].set_xlabel('Years Since 1980')
    axs[1].grid(True)
    axs[1].legend()

    plt.suptitle("Model 7 (Upgraded): Dynamic Growth Accounting by Decade", fontsize=14)
    plt.tight_layout()
    plt.show()

interact(
    dynamic_growth_accounting,
    alpha=FloatSlider(value=0.33, min=0.2, max=0.6, step=0.01, description='Capital Share α'),
    K80=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Init K 1980'),
    L80=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Init L 1980'),
    A80=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Init A 1980'),
    Kg1=FloatSlider(value=0.03, min=0, max=0.06, step=0.005, description='K Growth 80s'),
    Lg1=FloatSlider(value=0.02, min=0, max=0.05, step=0.005, description='L Growth 80s'),
    Ag1=FloatSlider(value=0.01, min=0, max=0.03, step=0.001, description='TFP 80s'),
    Kg2=FloatSlider(value=0.05, min=0, max=0.06, step=0.005, description='K Growth 90s'),
    Lg2=FloatSlider(value=0.01, min=0, max=0.05, step=0.005, description='L Growth 90s'),
    Ag2=FloatSlider(value=0.015, min=0, max=0.03, step=0.001, description='TFP 90s'),
    Kg3=FloatSlider(value=0.02, min=0, max=0.06, step=0.005, description='K Growth 00s'),
    Lg3=FloatSlider(value=0.01, min=0, max=0.05, step=0.005, description='L Growth 00s'),
    Ag3=FloatSlider(value=0.007, min=0, max=0.03, step=0.001, description='TFP 00s')
)

interactive(children=(FloatSlider(value=0.33, description='Capital Share α', max=0.6, min=0.2, step=0.01), Flo…

<function __main__.dynamic_growth_accounting(alpha=0.33, K80=1.0, L80=1.0, A80=1.0, Kg1=0.03, Lg1=0.02, Ag1=0.01, Kg2=0.05, Lg2=0.01, Ag2=0.015, Kg3=0.02, Lg3=0.01, Ag3=0.007)>

# 🔍 Model 7: Dynamic Growth Accounting by Decade (Revised)

This model simulates capital, labor, TFP, and output growth from 1980–2010 in continuous time.

Each decade has its own assumed growth rates:

- Capital: g_K
- Labor: g_L
- Productivity: g_A

# 📐 Output Function:
\[
Y_t = K_t^{\alpha} (A_t \cdot L_t)^{1 - \alpha}
\]

# 💡 Insight:
Instead of static bars, we now track how capital, labor, and TFP contributions **accumulate over time** in logs — just like growth economists analyze in real data.

Use this to:
- Model real-world transitions (e.g. East Asia’s growth, post-2008 slowdown)
- Show how **TFP can lag** while capital surges
- Tie your simulation to **balanced growth theory**

# 📊 Growth Accounting by Decade

This model breaks total output growth across three decades into:

- Capital contribution (α × g_K)
- Labor contribution ((1 – α) × g_L)
- TFP growth (g_A)

# Equation:
\[
\frac{\dot{Y}}{Y} = \alpha \frac{\dot{K}}{K} + (1 - \alpha) \frac{\dot{L}}{L} + \frac{\dot{A}}{A}
\]

This helps identify:
- **Which factor drove growth** in each decade
- When **productivity surged or stalled**
- How growth transitions over time

> Try adjusting TFP in the 2000s to simulate slowing growth.